<a href="https://colab.research.google.com/github/ryan-aday/HuggingFace_DL/blob/main/HuggingFace_Cert_Unit1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Based on: https://huggingface.co/docs/huggingface_hub/v0.16.2/en/guides/upload

#Install all dependencies
# If broken, follow this: https://stackoverflow.com/questions/76222239/pip-install-gymnasiumbox2d-not-working-on-google-colab

!pip install wheel setuptools pip --upgrade
!pip install --upgrade pip
!pip install swig
!pip install gymnasium[box2d]
!apt install cmake
!pip install stable-baselines3==2.0.0a5
!pip install huggingface_sb3

!apt install python3-opengl
!apt install ffmpeg
!apt install xvfb
!pip install pyvirtualdisplay

!sudo apt-get update

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 37.5 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.7 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'box2d-py' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'box2d-py'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp312-cp312-linux_x86_64.whl size=2409569 sha256=6015bf854bb47a999fefeab521ee1b29f5b907e134a32e7805745331a3d9d3ee
  Stored in directory: /root/.cache/pip/wheels/2a/e9/60/774da0bcd07f7dc7761a8590fa2d065e4069568e78dcdc3318
Successfully built box2d-py
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newe

In [ ]:
# Sample block (No need to run)

import gymnasium as gym
from stable-baselines3.common.env_util import make_vec_env

# First, we create our environment called LunarLander-v2
env = gym.make("LunarLander-v2")

# Then we reset this environment
observation, info = env.reset()

for _ in range(20):
    # Take a random action
    action = env.action_space.sample()
    print("Action taken:", action)

    # Do this action in the environment and get
    # next_state, reward, terminated, truncated and info
    observation, reward, terminated, truncated, info = env.step(action)

    # If the game is terminated (in our case we land, crashed) or truncated (timeout)
    if terminated or truncated:
        # Reset the environment
        print("Environment is reset")
        observation, info = env.reset()

env.close()

Action taken: 1
Action taken: 3
Action taken: 0
Action taken: 0
Action taken: 0
Action taken: 1
Action taken: 3
Action taken: 0
Action taken: 1
Action taken: 3
Action taken: 3
Action taken: 3
Action taken: 0
Action taken: 2
Action taken: 1
Action taken: 2
Action taken: 2
Action taken: 3
Action taken: 1
Action taken: 3


In [ ]:
# View environment data

# We create our environment with gym.make("<name_of_the_environment>")
env = gym.make("LunarLander-v2")
env.reset()
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample())  # Get a random observation

print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("""
  Action 0: Do nothing,
  Action 1: Fire left orientation engine,
  Action 2: Fire the main engine,
  Action 3: Fire right orientation engine.
  """)
print("Action Space Sample", env.action_space.sample())  # Take a random action



_____OBSERVATION SPACE_____ 

Observation Space Shape (8,)
Sample observation [ 37.15994    -31.923515    -4.855486    -1.4552459    2.5209994
  -1.2411039    0.68132234   0.581244  ]

 _____ACTION SPACE_____ 

Action Space Shape 4

  Action 0: Do nothing,
  Action 1: Fire left orientation engine,
  Action 2: Fire the main engine,
  Action 3: Fire right orientation engine.
  
Action Space Sample 2


In [ ]:
# Import necessary packages

# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import login

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

import gymnasium as gym

# Create the environment
env = make_vec_env("LunarLander-v2", n_envs=32)

# TODO: Define a PPO MlpPolicy architecture
# We use MultiLayerPerceptron (MLPPolicy) because the input is a vector,
# if we had frames as input we would use CnnPolic

# https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html

# Instantiate the agent
model = PPO(
    policy="MlpPolicy",
    env=env,
    learning_rate=0.0004,
    n_steps=2048,
    batch_size=64,
    n_epochs=4,
    gamma=0.999,
    gae_lambda=0.98,
    ent_coef=0.01,
    verbose=True,
    clip_range=0.3
)

# Train the agent (Need to train with 1e6 timesteps)
model.learn(total_timesteps=int(2e6))

# TODO: Specify file name for model and save the model to file
model_name = "ppo-LunarLander-v2"
model.save(model_name)

# Evaluate the model
eval_env = Monitor(gym.make("LunarLander-v2"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

Using cpu device


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 88.3     |
|    ep_rew_mean     | -183     |
| time/              |          |
|    fps             | 4009     |
|    iterations      | 1        |
|    time_elapsed    | 16       |
|    total_timesteps | 65536    |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 89.1          |
|    ep_rew_mean          | -123          |
| time/                   |               |
|    fps                  | 2547          |
|    iterations           | 2             |
|    time_elapsed         | 51            |
|    total_timesteps      | 131072        |
| train/                  |               |
|    approx_kl            | 0.019174382   |
|    clip_fraction        | 0.0929        |
|    clip_range           | 0.3           |
|    entropy_loss         | -1.37         |
|    explained_variance   | 1.3291836e-05 |


In [ ]:
login()

import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env

from huggingface_sb3 import package_to_hub

# PLACE the variables you've just defined two cells above
# Define the name of the environment
env_id = "LunarLander-v2"

# TODO: Define the model architecture we used
model_architecture = "PPO"

## Define a repo_id
## repo_id is the id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
## CHANGE WITH YOUR REPO ID
repo_id = "RyanObelisk/ppo-LunarLander-v2"  # Change with your repo id, you can't push with mine 😄

## Define the commit message
commit_message = "Upload PPO LunarLander-v2 trained agent"

# Create the evaluation env and set the render_mode="rgb_array"
eval_env = DummyVecEnv([lambda: Monitor(gym.make(env_id, render_mode="rgb_array"))])

# PLACE the package_to_hub function you've just filled here
package_to_hub(
    model=model,  # Our trained model
    model_name=model_name,  # The name of our trained model
    model_architecture=model_architecture,  # The model architecture we used: in our case PPO
    env_id=env_id,  # Name of the environment
    eval_env=eval_env,  # Evaluation Environment
    repo_id=repo_id,  # id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
    commit_message=commit_message,
)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.
Saving video to /tmp/tmpbzsj3zum/-step-0-to-step-1000.mp4


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"


Moviepy - Building video /tmp/tmpbzsj3zum/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmpbzsj3zum/-step-0-to-step-1000.mp4



/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Moviepy - Done !
Moviepy - video ready /tmp/tmpbzsj3zum/-step-0-to-step-1000.mp4
ℹ Pushing repo RyanObelisk/ppo-LunarLander-v2 to the Hugging Face
Hub


...

  ...ct1wq/ppo-LunarLander-v2/policy.pth: 100%|##########| 44.0kB / 44.0kB            

  .../tmpg38ct1wq/ppo-LunarLander-v2.zip: 100%|##########|  149kB /  149kB            

  /tmp/tmpg38ct1wq/replay.mp4           : 100%|##########|  151kB /  151kB            

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/RyanObelisk/ppo-LunarLander-v2/tree/main/


CommitInfo(commit_url='https://huggingface.co/RyanObelisk/ppo-LunarLander-v2/commit/228f9d7133683832b241aa84aa34f267fd9757da', commit_message='Upload PPO LunarLander-v2 trained agent', commit_description='', oid='228f9d7133683832b241aa84aa34f267fd9757da', pr_url=None, repo_url=RepoUrl('https://huggingface.co/RyanObelisk/ppo-LunarLander-v2', endpoint='https://huggingface.co', repo_type='model', repo_id='RyanObelisk/ppo-LunarLander-v2'), pr_revision=None, pr_num=None)